In [18]:
import json
import pathlib
from pprint import pprint
import sys

import openai
import pandas as pd

from matplotlib.pyplot as plt

from judge_gpt.auth import auth_gcloud, set_openai_apikey_from_file
from judge_gpt.conf import read_conf
from judge_gpt.tools import make_inputs_from_df

## 認証類

In [4]:
# 設定ファイル
conf_judge, conf_response_schema = read_conf(
    "./conf/judge.yaml",
    "./conf/response_schema.yaml",
)

# 認証
gc = auth_gcloud(conf_judge["gss"]["gcloud"])
set_openai_apikey_from_file(conf_judge["chatgpt"]["secrets"])

## 読み込み

In [6]:
wb = gc.open_by_key(conf_judge["gss"]["gss_key"])
ws = wb.get_worksheet(0)

ws_values = ws.get_values()
df_ws = pd.DataFrame(ws_values[1:], columns=ws_values[0])

display(df_ws)

,タイムスタンプ,回答者,演者,難易度,多彩性度,操作安定度,演技構成
0,2024/01/04 0:08:16,Alice,アリス,1ディアボロでは難易度の高い技をしているが、1ディアボロの技しか行っていなかった。,ボディ系の技とシンプルなインテグラルのみで幅としてはイマイチだった。,基本的に綺麗に技を行なっているが、たまにディアボロに引きずられていることがあった。,演技中のキャラ付けと曲と技の感じが良かった。
1,2024/01/04 0:08:57,Alice,チャーリー,繊細なコントロールが要求されるナンバーズのコラム・ハイトス系統が演技のほとんどを占めており、...,コラム・ハイトス系統がほとんどを占める演技であったが、様々な軌道でディアボロを動かすことで多...,よどみない操作かつミスのない演技で、非常に安定していた。ディアボロが喜んでいた。,黒を基調とした衣装のため、赤いディアボロが映えていた。序盤から2ディアボロをスタートする構成...


In [7]:
judge_names = df_ws["回答者"].unique()
print(judge_names)

['Alice']


In [8]:
judge_name = "Alice"
comment = make_inputs_from_df(df_ws.query(f"回答者 == '{judge_name}'"))

print(comment)

演者:アリス
難易度:1ディアボロでは難易度の高い技をしているが、1ディアボロの技しか行っていなかった。
多彩性度：ボディ系の技とシンプルなインテグラルのみで幅としてはイマイチだった。
操作安定度:基本的に綺麗に技を行なっているが、たまにディアボロに引きずられていることがあった。
演技構成度:演技中のキャラ付けと曲と技の感じが良かった。
演者:チャーリー
難易度:繊細なコントロールが要求されるナンバーズのコラム・ハイトス系統が演技のほとんどを占めており、極めて高難易度な演技であった。
多彩性度：コラム・ハイトス系統がほとんどを占める演技であったが、様々な軌道でディアボロを動かすことで多彩度を出せていた
操作安定度:よどみない操作かつミスのない演技で、非常に安定していた。ディアボロが喜んでいた。
演技構成度:黒を基調とした衣装のため、赤いディアボロが映えていた。序盤から2ディアボロをスタートする構成も見ごたえがあり非常に良かった。


## 実際に投げてみる

In [9]:
client = openai.OpenAI()

In [14]:
message = [
    {"role": "system", "content": conf_judge["chatgpt"]["prompt_judge"]},
] + [
    {"role": "user", "content": comment},
]

pprint(message)

[{'content': 'これからあなたにはジャグリングのパフォーマンスに関する評価コメントを読んでそれぞれの評価コメントを点数化してください。\n'
             'コメントは難易度、多彩性度、操作安定度、新奇性度、演技構成度の5つの観点で記載されています。それぞれの観点の定義と配点は以下の通りです。\n'
             '- 難易度（配点 35点）：技の要素における、難易度、数を評価する。\n'
             '- 多彩性度（配点 15点）：演技中に行った技が多彩性に富んでいたかを評価する。\n'
             '- 操作安定度（配点 25点）：技の実施において、巧みさ、美しさ、洗練度合いを評価する。\n'
             '- 演技構成度（配点 25点）：演技開始から演技終了までにおいて、楽曲、衣装、技の順序、起承転結などの調和性を評価する。\n'
             'これから一人の審査員のそれぞれの演者に対するコメントをもとに渡します。演者ごとに比較し順位をつけることを意識して、各項目の配点内で採点してください。\n',
  'role': 'system'},
 {'content': '演者:アリス\n'
             '難易度:1ディアボロでは難易度の高い技をしているが、1ディアボロの技しか行っていなかった。\n'
             '多彩性度：ボディ系の技とシンプルなインテグラルのみで幅としてはイマイチだった。\n'
             '操作安定度:基本的に綺麗に技を行なっているが、たまにディアボロに引きずられていることがあった。\n'
             '演技構成度:演技中のキャラ付けと曲と技の感じが良かった。\n'
             '演者:チャーリー\n'
             '難易度:繊細なコントロールが要求されるナンバーズのコラム・ハイトス系統が演技のほとんどを占めており、極めて高難易度な演技であった。\n'
             '多彩性度：コラム・ハイトス系統がほとんどを占める演技であったが、様々な軌道でディアボロを動かすことで多彩度を出せていた\n'
             '操作安定度:よどみない操作かつミスの

In [15]:
res = client.chat.completions.create(
    model=conf_judge["chatgpt"]["model"],
    messages=message,
    tools=[conf_response_schema],
    tool_choice="auto",
    seed=43,
)

In [16]:
res_message = res.choices[0].message
# print(res_message)
# tool_calls = res_message.tool_calls

In [19]:
judge_result = json.loads(
    res_message.tool_calls[0].function.arguments
)

pprint(judge_result)

judge_result = pd.DataFrame(judge_result["judge_results"])

{'judge_results': [{'cleanliness': 3,
                    'desc_cleanliness': '基本的に綺麗に技を行なっているが、たまにディアボロに引きずられていることがあった。',
                    'desc_difficulty': '1ディアボロでは難易度の高い技をしているが、1ディアボロの技しか行っていなかった。',
                    'desc_program_component': '演技中のキャラ付けと曲と技の感じが良かった。',
                    'desc_variation': 'ボディ系の技とシンプルなインテグラルのみで幅としてはイマイチだった。',
                    'difficulty': 5,
                    'player_name': 'アリス',
                    'program_component': 4,
                    'variation': 2},
                   {'cleanliness': 9,
                    'desc_cleanliness': 'よどみない操作かつミスのない演技で、非常に安定していた。ディアボロが喜んでいた。',
                    'desc_difficulty': '繊細なコントロールが要求されるナンバーズのコラム・ハイトス系統が演技のほとんどを占めており、極めて高難易度な演技であった。',
                    'desc_program_component': '黒を基調とした衣装のため、赤いディアボロが映えていた。序盤から2ディアボロをスタートする構成も見ごたえがあり非常に良かった。',
                    'desc_variation': 'コラム・ハイトス系統がほとんどを占める演技であったが、様々な軌道でディアボロを動かすことで多彩度を出せていた',
                    'difficulty': 9,
              

In [16]:
judge_result["description"] = judge_result["desc_difficulty"].str.cat(
    [
        judge_result["desc_variation"], 
        judge_result["desc_cleanliness"],
        judge_result["desc_program_component"],
    ],
    sep="\n",
)

judge_result = judge_result.drop(columns=[
    col for col in judge_result.columns if col.startswith("desc_")
])


In [20]:
display(judge_result)

,player_name,difficulty,desc_difficulty,variation,desc_variation,cleanliness,desc_cleanliness,program_component,desc_program_component
0,アリス,5,1ディアボロでは難易度の高い技をしているが、1ディアボロの技しか行っていなかった。,2,ボディ系の技とシンプルなインテグラルのみで幅としてはイマイチだった。,3,基本的に綺麗に技を行なっているが、たまにディアボロに引きずられていることがあった。,4,演技中のキャラ付けと曲と技の感じが良かった。
1,チャーリー,9,繊細なコントロールが要求されるナンバーズのコラム・ハイトス系統が演技のほとんどを占めており、...,8,コラム・ハイトス系統がほとんどを占める演技であったが、様々な軌道でディアボロを動かすことで多...,9,よどみない操作かつミスのない演技で、非常に安定していた。ディアボロが喜んでいた。,8,黒を基調とした衣装のため、赤いディアボロが映えていた。序盤から2ディアボロをスタートする構成...


In [21]:
fig, ax = plt.subplots()

judge_result.set_index("player_name").plot(kind="bar", ax=ax)
ax.grid()

NameError: name 'plt' is not defined